# PS1 HST Project Intro

This is a brief tutorial for inspecting HST and Pan-STARRS images through Python.  In addition to these steps, DS9 (http://ds9.si.edu/site/Home.html) is extremely useful software for viewing and manipulating images.  I'll put up an additional tutorial for DS9 soon.

Astronomical images are in FITS format, which stands for Flexible Image Transport System.  In Python, the best way to read them is using the Astropy package.  Assuming you've transferred the HST and Pan-STARRS test data from Ziggy, you can start reading in the data.

### Getting Started

In [ ]:
# import a few standard packages
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
%matplotlib inline
# this last line shows plots in the notebook

In [ ]:
hdu = fits.open('370330/acs_f814w_drz.fits')
header = hdu[0].header
data = hdu[0].data

FITS images have header information and then the actual image data.  Let's check out the header first.  A lot of this information is not necessary to understand, but in short the header defines the coordinates of the image on the sky, the filter (color) through which the image was taken, and a number of other useful things.

In [ ]:
header

Next, let's check out the data, which is formatted as a numpy array.

In [ ]:
data

In [ ]:
# vmin and vmax are just from trial and error, for now
plt.imshow(data,vmin=-0.05,vmax=100.0,cmap="gray")

### Inspecting the Host Galaxy of the Supernova

To find out where the supernova is, we start with its position on the sky.  Astronomical objects are identified by two coordinates, right ascension and declination (RA and Dec).  Coordinates are given by the file in the git repo, mds_sne.txt

In [ ]:
snid,ra,dec = np.loadtxt('mds_sne.txt',unpack=True,dtype="U10,f8,f8")
print(ra[snid == '370330'][0],dec[snid == '370330'][0])

Now we have the RA/Dec, and we use Astropy to convert into x,y coordinates of the image.

In [ ]:
from astropy.wcs import WCS
hstwcs = WCS('370330/acs_f814w_drz.fits')
coords = hstwcs.wcs_world2pix([(ra[snid == '370330'][0],dec[snid == '370330'][0])],0)
x_sn, y_sn = coords[0]
print("our SN position is (x,y) = %.2f,%.2f"%(x_sn,y_sn))

Notice that Charlie has pre-processed these images so that they are centered on the supernova already!  Now, let's take a look at the SN host galaxy by plotting a cutout of the image array.  Don't forget that in python, the first index is Y and the *second* index is X.  

Last important but annoying thing - in Python, array[0:10] selects array elements 0-9.  So there's a +1 at the end of my indexes.  And indexes must be integers, of course.

If you play with the vmax key, you can see more or less of the galaxy.  I'm using a low vmax value so that you can see that there is a little bit of galaxy flux at the SN location.

In [ ]:
half_box_size = 100
subim = data[int(y_sn-half_box_size):int(y_sn+half_box_size+1),
             int(x_sn-half_box_size):int(x_sn+half_box_size+1)]
plt.imshow(subim,vmin=-0.05,vmax=700)
plt.plot(half_box_size,half_box_size,'+',ms=20,color='r')

### Checking out the Pixels

The pixels of the image themselves are calibrated to give you real fluxes in some (for now) arbitrary units.  Let's see how bright the SN pixel is compared to the other pixels.

In [ ]:
plt.hist(subim.flatten())
plt.yscale('log')
plt.axvline(subim[half_box_size,half_box_size],color='r')
plt.xlabel('Pixel Value')
plt.ylabel('Number')
print('Pixel at the SN location has host galaxy flux %.3f'%subim[half_box_size,half_box_size])

What if we only want pixels that are part of the galaxy, rather than the background?  For that we need to figure out what the approximate size of the galaxy really is.  Let's tackle that later :).